In [14]:
 # Short aliases for readability
KEY_ALIASES = {
        # dataset
        "dataset.num_samples": "n",
        "dataset.num_series": "d",
        "dataset.num_features": "d",
        "dataset.seq_len": "L",
        "dataset.window_size": "w",
        "dataset.num_interactions": "m",
        "dataset.autocorr_coeff": "gamma",
        
        "dataset.cross_coef": "alpha",
        "dataset.trend_window_size": "tws",
        "dataset.label_mode": "lbl",
        # model
        "model.name": "m",
        "model.d_model": "dm",
        "model.hidden": "h",
        "model.layers": "ly",
        "model.n_heads": "H",

        # training
        "training.epochs": "ep",
        "training.batch_size": "bs",
        "training.lr": "lr",
        "training.weight_decay": "wd",


        # pointwise

        "pointwise.explainer.name": "xai",
        "pointwise.explainer.params.steps": "steps",

        # pairwise
        "pairwise.interaction_method.name": "im",
        "pairwise.tau_max": "taumax",
        "pairwise.K": "K",
        "pairwise.batch_size": "bsval",
        "pairwise.params.num_permutations": "nperm"
    }

import json

def load_pointwise_metrics(run_dir):
    """
    Load pointwise metrics if present.
    Returns flat dict with prefixed keys.
    """
    out = {}

    std_path = run_dir / "pointwise_metrics.json"
    hug_path = run_dir / "pointwise_metrics_hugues.json"

    if std_path.exists():
        with open(std_path) as f:
            d = json.load(f)
        for k, v in d.items():
            out[f"pointwise.{k}"] = v

    if hug_path.exists():
        with open(hug_path) as f:
            d = json.load(f)
        for k, v in d.items():
            out[f"pointwise.hug_{k}"] = v

    return out



In [15]:
import os
import re
import json
from pathlib import Path
from collections import defaultdict
import pandas as pd
import numpy as np

RUNS_ROOT = Path("../runs_new")

DATASETS_OF_INTEREST = {"var_local", "var_nonlocal"}  # you said focus first two

# ----------------------------
# Helpers: safe IO
# ----------------------------
def safe_read_json(p: Path):
    try:
        with open(p, "r") as f:
            return json.load(f)
    except Exception:
        return None

def safe_glob_one(d: Path, pattern: str):
    xs = list(d.glob(pattern))
    return xs[0] if xs else None

# ----------------------------
# Parsing utilities
# ----------------------------
# captures tokens like: a1_0.2, noise=0.1, d_model=64, tau_max25 (partial handled below)
KV_RE = re.compile(r"(?P<key>[A-Za-z][A-Za-z0-9]*)[=_](?P<val>[-+]?\d*\.?\d+(?:e[-+]?\d+)?)")

# captures tokens like: tau25, perm20, ep100, bs32 etc.
# PREFIXNUM_RE = re.compile(r"^(?P<key>[A-Za-z]+)(?P<val>[-+]?\d*\.?\d+(?:e[-+]?\d+)?)$")
PREFIXNUM_RE = re.compile(
    r"^(?P<key>[A-Za-z]+)(?P<val>[-+]?\d*\.?\d+(?:e[-+]?\d+)?)$"
)


def cast_num(s: str):
    # int if clean else float
    try:
        if re.fullmatch(r"[-+]?\d+", s):
            return int(s)
        return float(s)
    except Exception:
        return s

def parse_params_from_parts(parts):
    """
    Parse params from path components using:
    - key=value or key_value
    - prefix+number tokens (bs32, ep100, tau25, perm20)
    """
    params = {}
    for part in parts:
        # split further on common delimiters
        subtoks = re.split(r"[,\s]+", part)
        for tok in subtoks:
            if not tok:
                continue

            # key=val or key_val
            m = KV_RE.search(tok)
            if m:
                k = m.group("key")
                v = cast_num(m.group("val"))
                params[k] = v
                continue

            # prefix+number (e.g., bs32, ep100, tau25, perm20)
            m2 = PREFIXNUM_RE.match(tok)
            if m2:
                k = m2.group("key")
                v = cast_num(m2.group("val"))
                # only accept if key looks like known run hyperparam prefixes
                ALLOWED_PREFIX_KEYS = {
                    # dataset
                    "a", "n", "d", "L", "w", "m", "gamma", "alpha", "tws",
                    # model
                    "dm", "h", "ly", "H",
                    # training
                    "bs", "ep", "lr", "wd",
                    # xai
                    "steps",
                    # pairwise
                    "taumax", "K", "nperm",
                }

                if k in ALLOWED_PREFIX_KEYS:
                    params[k] = v

    return params

def find_dataset_in_path(parts):
    for p in parts:
        pl = p.lower()
        if pl in {"var_local", "var_nonlocal", "cltts"}:
            return pl
    return None

def find_model_in_path(parts):
    # add more model names if needed
    known = {"transformer", "lstm", "tcn"}
    for p in parts:
        pl = p.lower()
        if pl in known:
            return pl
    return None

def infer_interaction_method(run_dir: Path):
    # your pipeline: interaction_curves_{method}.pkl
    for p in run_dir.glob("interaction_curves_*.pkl"):
        name = p.stem  # interaction_curves_ih
        m = name.split("interaction_curves_")[-1]
        return m
    return None

# ----------------------------
# Identify run directories
# ----------------------------
POINTWISE_EXPLAINERS = {"ig", "deeplift", "gradshap"}


RUN_MARKERS = [
    "pairwise_faithfulness_metrics.json",
    "pointwise_metrics.json",
    "history.json",
    "metrics1.json",
]

def is_run_dir(d: Path):
    if not d.is_dir():
        return False
    for mk in RUN_MARKERS:
        if (d / mk).exists():
            return True
    # also treat presence of interaction_curves_*.pkl as run dir
    if list(d.glob("interaction_curves_*.pkl")):
        return True
    return False

def collect_run_dirs(root: Path):
    run_dirs = []
    for dirpath, dirnames, filenames in os.walk(root):
        d = Path(dirpath)
        if is_run_dir(d):
            run_dirs.append(d)
    return sorted(set(run_dirs))



# ----------------------------
# Extract a row from a run dir
# ----------------------------
def extract_run_record(run_dir: Path):
    parts = run_dir.parts

    rec = {
        "run_dir": str(run_dir),
        "dataset": find_dataset_in_path(parts),
        "model": find_model_in_path(parts),
        "interaction_method": infer_interaction_method(run_dir),
    }

    # 1) If meta.json exists, trust it most
    meta = safe_read_json(run_dir / "meta.json")
    if meta:
        # attempt common schemas
        # meta might contain nested cfg, dataset, model, etc.
        rec["meta_present"] = True

        # dataset
        if "dataset" in meta and isinstance(meta["dataset"], dict):
            rec["dataset"] = rec["dataset"] or meta["dataset"].get("name")
            # copy scalar dataset params
            for k, v in meta["dataset"].items():
                if isinstance(v, (int, float, str, bool)):
                    rec[f"dataset.{k}"] = v

        # model
        if "model" in meta and isinstance(meta["model"], dict):
            rec["model"] = rec["model"] or meta["model"].get("name")
            for k, v in meta["model"].items():
                if isinstance(v, (int, float, str, bool)):
                    rec[f"model.{k}"] = v

        # pairwise / xai params
        for topk in ["pairwise", "pointwise", "pointwise_xai"]:
            if topk in meta and isinstance(meta[topk], dict):
                for k, v in meta[topk].items():
                    if isinstance(v, (int, float, str, bool)):
                        rec[f"{topk}.{k}"] = v

    else:
        rec["meta_present"] = False

    # 2) Parse remaining params from directory tokens (works even without meta)
    parsed = parse_params_from_parts(parts)

    # Normalize a few likely keys to your naming
    key_map = {
        "taumax": "tau_max",
        "tau": "tau_max",
        "perm": "num_permutations",
        "bs": "batch_size",
        "ep": "epochs",
    }
    for k, v in parsed.items():
        nk = key_map.get(k, k)
        rec[nk] = v

    # 3) Load metrics (if exist)
    pw = safe_read_json(run_dir / "pointwise_metrics.json")
    if pw:
        for k, v in pw.items():
            if isinstance(v, (int, float, str, bool)):
                rec[f"pointwise.{k}"] = v

    pair = safe_read_json(run_dir / "pairwise_faithfulness_metrics.json")
    if pair:
        # flatten a few important ones
        gr = pair.get("graph_recovery", {})
        if "lag_sensitive" in gr:
            for k, v in gr["lag_sensitive"].items():
                if isinstance(v, (int, float)):
                    rec[f"pairwise.ls.{k}"] = float(v)
        if "lag_agnostic" in gr:
            for k, v in gr["lag_agnostic"].items():
                if isinstance(v, (int, float)):
                    rec[f"pairwise.la.{k}"] = float(v)

        ll = pair.get("lag_locality", {})
        for k, v in ll.items():
            if isinstance(v, (int, float)):
                rec[f"pairwise.lag_locality.{k}"] = float(v)

        sc = pair.get("strength_calibration", {})
        for k, v in sc.items():
            if isinstance(v, (int, float)):
                rec[f"pairwise.strength.{k}"] = float(v)

        if "threshold_delta" in gr:
            rec["pairwise.delta"] = gr["threshold_delta"]

    hist = safe_read_json(run_dir / "history.json")
    if hist and isinstance(hist, list) and len(hist) > 0:
        last = hist[-1]
        for k, v in last.items():
            if isinstance(v, (int, float)):
                rec[f"clf.{k}"] = float(v)

    # --------------------------------------------------
    # Pointwise metrics (IG / DeepLIFT / GradSHAP)
    # --------------------------------------------------
    if rec.get("pointwise.explainer.name") in POINTWISE_EXPLAINERS:
        pm = load_pointwise_metrics(run_dir)
        rec.update(pm)


    # 4) Small convenience: bring common dataset params into top-level if present
    for k in ["a1", "noise", "rho", "gamma", "window_size"]:
        if k not in rec:
            # try dataset.a1 style
            if f"dataset.{k}" in rec:
                rec[k] = rec[f"dataset.{k}"]

    return rec

def apply_key_aliases(rec, key_aliases):
    out = {}
    for k, v in rec.items():
        alias = key_aliases.get(k, k)
        out[alias] = v
    return out



# ----------------------------
# Hyperparam discovery printer
# ----------------------------
def print_discovered_hyperparams(df, keys=None):
    if keys is None:
        # smart default: show only keys that vary and are "param-like"
        candidate = []
        for c in df.columns:
            if c in {"run_dir"}:
                continue
            if any(c.endswith(suf) for suf in [".name"]) or c.startswith(("dataset.", "model.", "pairwise.", "pointwise.")) or c in {
                "dataset","model","interaction_method","a1","noise","tau_max","num_permutations","d_model","layers","hidden","batch_size","epochs"
            }:
                candidate.append(c)
        keys = candidate

    print("🔎 Discovered hyperparams in run dirs:")
    for k in sorted(set(keys)):
        if k not in df.columns:
            continue
        vals = df[k].dropna().unique()
        if len(vals) <= 1:
            continue
        # sort nicely (numbers then strings)
        try:
            vals_sorted = sorted(vals, key=lambda x: (isinstance(x, str), x))
        except Exception:
            vals_sorted = list(vals)
        print(f"  {k}: {vals_sorted}")


In [16]:

# ============================
# MAIN
# ============================
run_dirs = collect_run_dirs(RUNS_ROOT)
print(f"Found {len(run_dirs)} run dirs under {RUNS_ROOT}")

Found 272 run dirs under ../runs_new


In [19]:
df

,run_dir,dataset,model,interaction_method,meta_present,dataset.name,dataset.all_times,n,L,w,...,d,dataset.a1,dataset.split_ratio,a1,pointwise.AOPC_deletion,pointwise.AOPC_insertion,pointwise.Sufficiency,pointwise.Comprehensiveness,dataset.a9,dataset.a10
149,../runs_new/var_local_a10.4_n400_d3_L50_split_...,var_local,transformer,None,True,var_local,False,400.0,50.0,NaN,...,3.0,0.40,0.8,0.40,NaN,NaN,NaN,NaN,NaN,NaN
155,../runs_new/var_local_a10.4_n400_d3_L50_split_...,var_local,transformer,None,True,var_local,False,400.0,50.0,NaN,...,3.0,0.40,0.8,0.40,NaN,NaN,NaN,NaN,NaN,NaN
170,../runs_new/var_local_a10.8_n400_d3_L50_split_...,var_local,transformer,None,True,var_local,False,400.0,50.0,NaN,...,3.0,0.80,0.8,0.80,NaN,NaN,NaN,NaN,NaN,NaN
96,../runs_new/var_local_a10.05_n400_d3_L50_split...,var_local,transformer,None,True,var_local,False,400.0,50.0,NaN,...,3.0,0.05,0.8,0.05,NaN,NaN,NaN,NaN,NaN,NaN
81,../runs_new/var_local_a10.05_n400_d3_L50_split...,var_local,transformer,None,True,var_local,False,400.0,50.0,NaN,...,3.0,0.05,0.8,0.05,NaN,NaN,NaN,NaN,NaN,NaN
176,../runs_new/var_local_a10.8_n400_d3_L50_split_...,var_local,transformer,None,True,var_local,False,400.0,50.0,NaN,...,3.0,0.80,0.8,0.80,NaN,NaN,NaN,NaN,NaN,NaN
182,../runs_new/var_local_a10.8_n400_d3_L50_split_...,var_local,transformer,None,True,var_local,False,400.0,50.0,NaN,...,3.0,0.80,0.8,0.80,NaN,NaN,NaN,NaN,NaN,NaN
80,../runs_new/var_local_a10.05_n400_d3_L50_split...,var_local,transformer,None,True,var_local,False,400.0,50.0,NaN,...,3.0,0.05,0.8,0.05,NaN,NaN,NaN,NaN,NaN,NaN
128,../runs_new/var_local_a10.2_n400_d3_L50_split_...,var_local,transformer,None,True,var_local,False,400.0,50.0,NaN,...,3.0,0.20,0.8,0.20,NaN,NaN,NaN,NaN,NaN,NaN
161,../runs_new/var_local_a10.4_n400_d3_L50_split_...,var_local,transformer,None,True,var_local,False,400.0,50.0,NaN,...,3.0,0.40,0.8,0.40,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:


raw_records = [extract_run_record(d) for d in run_dirs]
records = [apply_key_aliases(r, KEY_ALIASES) for r in raw_records]
df = pd.DataFrame(records)



# Focus on var_local / var_nonlocal (as requested)
df = df[df["dataset"].isin(DATASETS_OF_INTEREST)].copy()

# Optional: sort by dataset then AUPRC (lag-sensitive if available)
sort_cols = [c for c in ["dataset", "pairwise.ls.auprc", "clf.auprc", "clf.auroc"] if c in df.columns]
if sort_cols:
    df = df.sort_values(sort_cols, ascending=[True] + [False]*(len(sort_cols)-1))

print_discovered_hyperparams(df)

# Show a compact table first
show_cols = [
    "dataset", "model", "interaction_method",
    "a1", "noise", "tau_max", "num_permutations",
    "model.d_model", "model.layers", "model.hidden",
    "clf.f1", "clf.auroc", "clf.auprc",
    "pairwise.ls.f1", 
    "pairwise.ls.auroc", 
    "pairwise.ls.auprc",
    "pairwise.la.f1", 
    "pairwise.la.auroc", 
    "pairwise.la.auprc",
    "pairwise.lag_locality.mean_lag_error",
    "pairwise.strength.spearman",
    "run_dir",
]
show_cols = [c for c in show_cols if c in df.columns]

display(df[show_cols].reset_index(drop=True).head(50))

# Save for LaTeX / Excel
out_tsv = "results_summary_var.tsv"
df.to_csv(out_tsv, sep="\t", index=False)
print(f"✅ Saved full table to {out_tsv}")


🔎 Discovered hyperparams in run dirs:
  a1: [0.05, 0.1, 0.2, 0.4, 0.8]
  dataset: ['var_local', 'var_nonlocal']
  dataset.a1: [0.05, 0.1, 0.2, 0.4, 0.8]
  dataset.a9: [0.5, 0.6, 0.7, 0.8]
  dataset.name: ['var_local', 'var_nonlocal']
  model: ['lstm', 'transformer']


,dataset,model,interaction_method,a1,noise,clf.f1,clf.auroc,clf.auprc,pairwise.ls.f1,pairwise.ls.auroc,pairwise.ls.auprc,pairwise.la.f1,pairwise.la.auroc,pairwise.la.auprc,pairwise.lag_locality.mean_lag_error,pairwise.strength.spearman,run_dir
0,var_local,transformer,None,0.40,0.05,0.9647,1.0000,1.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,../runs_new/var_local_a10.4_n400_d3_L50_split_...
1,var_local,transformer,None,0.40,0.05,0.9722,1.0000,1.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,../runs_new/var_local_a10.4_n400_d3_L50_split_...
2,var_local,transformer,None,0.80,0.05,0.9859,0.9987,0.9985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,../runs_new/var_local_a10.8_n400_d3_L50_split_...
3,var_local,transformer,None,0.05,0.05,0.8667,0.9969,0.9968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,../runs_new/var_local_a10.05_n400_d3_L50_split...
4,var_local,transformer,None,0.05,0.05,0.9783,0.9955,0.9966,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,../runs_new/var_local_a10.05_n400_d3_L50_split...
5,var_local,transformer,None,0.80,0.05,0.9655,0.9949,0.9960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,../runs_new/var_local_a10.8_n400_d3_L50_split_...
6,var_local,transformer,None,0.80,0.05,0.9412,0.9923,0.9909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,../runs_new/var_local_a10.8_n400_d3_L50_split_...
7,var_local,transformer,None,0.05,0.05,0.9091,0.9905,0.9906,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,../runs_new/var_local_a10.05_n400_d3_L50_split...
8,var_local,transformer,None,0.20,0.05,0.9351,0.9894,0.9895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,../runs_new/var_local_a10.2_n400_d3_L50_split_...
9,var_local,transformer,None,0.40,0.05,0.8506,0.9881,0.9874,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,../runs_new/var_local_a10.4_n400_d3_L50_split_...


✅ Saved full table to results_summary_var.tsv
